In [2]:
import pandas as pd
import quandl
import numpy as np
import mplfinance as mpf
#Setup Quandl
apiKey="TmwEsGGvzZL-Y9o2o_KZ"
quandl.ApiConfig.api_key = apiKey
df = quandl.get('CHRIS/CME_ES1', start_date='2010-01-01', end_date='2021-03-12')
df.rename(columns={"Last": "Close","Previous Day Open Interest":"Open Interest"},inplace = True)
df.drop(columns=['Change','Settle'],inplace = True)
df.drop(columns=["Open Interest"],inplace = True)

In [3]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2010-01-04,1113.75,1129.75,1113.25,1128.75,1291254.0
2010-01-05,1128.50,1133.00,1125.00,1132.25,1378593.0
2010-01-06,1132.00,1135.50,1127.25,1133.00,1259921.0
2010-01-07,1133.00,1138.75,1127.00,1137.50,1567025.0
2010-01-08,1137.25,1141.75,1131.00,1141.50,1527666.0


In [4]:
agg_dict = {'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Volume': 'sum'}

r_df = df.resample('W-Mon',label="left",closed="left").agg(agg_dict)

In [5]:
df = r_df

In [6]:
#Setup Zones
df['C']=(df['Close']+df['Low']+df['High'])/3
df['range']=df['High']-df['Low']
df['D']=df['C']+(df['range']/2)
df['B']=df['C']-(df['range']/2)
df['E']=df['C']+(df['range'])
df['A']=df['C']-(df['range'])
#Shift zones columns
#a['x2'] = a.x2.shift(1)
df['C']=df.C.shift(1)
df['D']=df.D.shift(1)
df['B']=df.B.shift(1)
df['E']=df.E.shift(1)
df['A']=df.A.shift(1)
df['range']=df.range.shift(1)

#SET CLOSE AND OPEN ZONES 	
conditions=[
    (df['Close']<= df['A']),
    (df['Close']<= df['B']) & (df['Close']> df['A']),
    (df['Close']<= df['C']) & (df['Close']> df['B']),
    (df['Close']<= df['D']) & (df['Close']> df['C']),
    (df['Close']<= df['E']) & (df['Close']> df['D']),
    (df['Close']> df['E'])
]
choices=['CL1','CL2','CL3','CL4','CL5','CL6']
df['CLZONE']=np.select(conditions,choices)
conditions=[
    (df['Open']<= df['A']),
    (df['Open']<= df['B']) & (df['Open']> df['A']),
    (df['Open']<= df['C']) & (df['Open']> df['B']),
    (df['Open']<= df['D']) & (df['Open']> df['C']),
    (df['Open']<= df['E']) & (df['Open']> df['D']),
    (df['Open']> df['E'])
]
choices=['OP1','OP2','OP3','OP4','OP5','OP6']
df['OPZONE']=np.select(conditions,choices)
df['CLZONE']=df.CLZONE.shift(1)
df = df.iloc[2:]
df['CLOP']=df['CLZONE']+df['OPZONE']
df.drop(columns=['CLZONE','OPZONE'],inplace = True)
#df['CLOP'].value_counts(normalize=True)

# Zones Reached
conditions=[
    (df['Low']<df['A'])
]
choices=[1]
df['Reach1']=np.select(conditions,choices)
conditions=[
    (df['High']>=df['B']) & (df['Low']<=df['A']),
    (df['Low']<df['B']) & (df['Low']>=df['A']),
    (df['High']<df['B']) & (df['High']>=df['A'])
]
choices=[1,1,1]
df['Reach2']=np.select(conditions,choices)
conditions=[
    (df['High']>=df['C']) & (df['Low']<=df['B']),
    (df['Low']<df['C']) & (df['Low']>=df['B']),
    (df['High']<df['C']) & (df['High']>=df['B'])
]
choices=[1,1,1]
df['Reach3']=np.select(conditions,choices)
conditions=[
    (df['High']>=df['D']) & (df['Low']<=df['C']),
    (df['Low']<df['D']) & (df['Low']>=df['C']),
    (df['High']<df['D']) & (df['High']>=df['C'])
]
choices=[1,1,1]
df['Reach4']=np.select(conditions,choices)
conditions=[
    (df['High']>=df['E']) & (df['Low']<=df['D']),
    (df['Low']<df['E']) & (df['Low']>=df['D']),
    (df['High']<df['E']) & (df['High']>=df['D'])
]
choices=[1,1,1]
df['Reach5']=np.select(conditions,choices)
conditions=[
    (df['High']>df['E'])
]
choices=[1]
df['Reach6']=np.select(conditions,choices)

# Zonas como Resistencia:
#

conditions=[
    (df['High']>df['A']) & (df['High']<(df['B']))
]
choices=[1]
df['RES2']=np.select(conditions,choices)
conditions=[
    (df['High']>df['B']) & (df['High']<(df['C']))
]
choices=[1]
df['RES3']=np.select(conditions,choices)
conditions=[
    (df['High']>df['C']) & (df['High']<(df['D']))
]
choices=[1]
df['RES4']=np.select(conditions,choices)
conditions=[
    (df['High']>df['D']) & (df['High']<(df['E']))
]
choices=[1]
df['RES5']=np.select(conditions,choices)
conditions=[
    (df['High']>df['E']) & (df['Close']<(df['E']))
]
choices=[1]
df['RES6']=np.select(conditions,choices)

# Zonas como Soporte:
#

conditions=[
    (df['Low']<(df['A'])) & (df['Close']>df['A'])
]
choices=[1]
df['SUP1']=np.select(conditions,choices)
conditions=[
    (df['Low']<(df['B'])) & (df['Low']>df['A'])
]
choices=[1]
df['SUP2']=np.select(conditions,choices)
conditions=[
    (df['Low']<(df['C'])) & (df['Low']>df['B'])
]
choices=[1]
df['SUP3']=np.select(conditions,choices)
conditions=[
    (df['Low']<(df['D'])) & (df['Low']>df['C'])
]
choices=[1]
df['SUP4']=np.select(conditions,choices)
conditions=[
    (df['Low']<(df['E'])) & (df['Low']>df['D'])
]
choices=[1]
df['SUP5']=np.select(conditions,choices)

# Bandas como zonas de soporte y resistencia
#
conditions=[
       (df['Low'] >= (df['C']-(df['range']*0.618))) & (df['Low']<= df['B'])
]
choices=[1]
df['SB1']=np.select(conditions,choices)
conditions=[
    (df['Low']>=(df['C']-(df['range']*1.382))) & (df['Low']<=df['A']) & (df['Close']>=df['A'])
]
choices=[1]
df['SB2']=np.select(conditions,choices)
conditions=[
    (df['High']<=(df['C']+(df['range']*0.618))) & (df['High']>=df['D']) 
]
choices=[1]
df['RB1']=np.select(conditions,choices)
conditions=[
    (df['High']<=(df['C']+(df['range']*1.382))) & (df['High']>=df['E']) & (df['Close']<=df['E']) 
]
choices=[1]
df['RB2']=np.select(conditions,choices)

#############
#	Creacion de matriz de probabilidades
###########

CLOP_combo=df['CLOP'].sort_values().unique()
prob=pd.DataFrame(index=CLOP_combo,columns=['count','Reach1','Reach2','Reach3','Reach4','Reach5','Reach6','RES2','RES3','RES4','RES5','RES6','SUP1','SUP2','SUP3','SUP4','SUP5', 'SB1', 'SB2', 'RB1', 'RB2'])
for i in CLOP_combo:
    data=df[df['CLOP']==i]
    probs_raw=data[['Reach1','Reach2','Reach3','Reach4','Reach5','Reach6','RES2','RES3','RES4','RES5','RES6','SUP1','SUP2','SUP3','SUP4','SUP5','SB1', 'SB2', 'RB1', 'RB2']]
    prob.loc[i,"count"]=data.shape[0]
    for c in probs_raw.columns:
        aux=probs_raw[c].value_counts(normalize=True)     
        if(1 in aux.index):
            prob.loc[i,c]=aux[1]*100.0
        else:
            prob.loc[i,c]=0.0
        if((c in ['RES1','SUP1','SB2']) & (prob.loc[i,'Reach1']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach1']*100.0
        if((c in ['RES2','SUP2','SB1']) & (prob.loc[i,'Reach2']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach2']*100.0
        if((c in ['RES3','SUP3']) & (prob.loc[i,'Reach3']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach3']*100.0
        if((c in ['RES4','SUP4']) & (prob.loc[i,'Reach4']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach4']*100.0
        if((c in ['RES5','SUP5','RB1']) & (prob.loc[i,'Reach5']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach5']*100.0
        if((c in ['RES6','SUP6','RB2']) & (prob.loc[i,'Reach6']!= 0)):
            prob.loc[i,c]=prob.loc[i,c]/prob.loc[i,'Reach6']*100.0
prob.round(2)

#############
#	Creacion de matriz de probabilidades Estrictas para bandas
###########

prob_strict=pd.DataFrame(index=CLOP_combo,columns=['RB1_s','RB2_s','SB1_s','SB2_s'])

prob_strict['RB1_s'] = np.divide(prob['RB1']*100, prob['RES5'], out=np.zeros_like(prob['RB1']), where=prob['RES5']!=0)
prob_strict['RB2_s'] = np.divide(prob['RB2']*100, prob['RES6'], out=np.zeros_like(prob['RB2']), where=prob['RES6']!=0)
prob_strict['SB1_s'] = np.divide(prob['SB1']*100, prob['SUP2'], out=np.zeros_like(prob['SB1']), where=prob['SUP2']!=0)
prob_strict['SB2_s'] = np.divide(prob['SB2']*100, prob['SUP1'], out=np.zeros_like(prob['SB2']), where=prob['SUP1']!=0)


<ipython-input-6-e2c00207ce55>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLOP']=df['CLZONE']+df['OPZONE']
K:\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-6-e2c00207ce55>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

In [7]:
prob_strict

,RB1_s,RB2_s,SB1_s,SB2_s
CL1OP2,0,0,0,100
CL1OP3,44.4444,100,43.75,50
CL1OP4,0,0,0,100
CL2OP2,0,0,0,100
CL2OP3,42.8571,100,33.3333,100
CL2OP4,0,0,0,0
CL3OP1,0,0,0,0
CL3OP2,0,0,0,0
CL3OP3,31.8182,100,25,66.6667
CL3OP4,16.6667,100,38.4615,0


In [8]:
prob

,count,Reach1,Reach2,Reach3,Reach4,Reach5,Reach6,RES2,RES3,RES4,...,RES6,SUP1,SUP2,SUP3,SUP4,SUP5,SB1,SB2,RB1,RB2
CL1OP2,1,100,100,100,100,0,0,0,0,100,...,0,100,0,0,0,0,0,100,0,0
CL1OP3,37,16.2162,59.4595,100,72.973,32.4324,8.10811,0,27.027,55.5556,...,33.3333,66.6667,72.7273,40.5405,0,0,31.8182,33.3333,33.3333,33.3333
CL1OP4,4,25,50,100,100,50,50,0,0,50,...,0,100,50,50,0,0,0,100,0,0
CL2OP2,2,50,100,100,100,0,0,0,0,100,...,0,100,50,0,0,0,0,100,0,0
CL2OP3,46,19.5652,58.6957,100,80.4348,50,4.34783,0,19.5652,37.8378,...,50,44.4444,66.6667,41.3043,0,0,22.2222,44.4444,39.1304,50
CL2OP4,5,40,40,60,100,40,20,0,0,60,...,0,50,0,33.3333,40,0,0,0,0,0
CL3OP1,1,100,100,0,0,0,0,100,0,0,...,0,100,0,0,0,0,0,0,0,0
CL3OP2,3,66.6667,100,66.6667,33.3333,0,0,33.3333,50,100,...,0,0,33.3333,0,0,0,0,0,0,0
CL3OP3,64,28.125,65.625,100,87.5,40.625,6.25,0,12.5,53.5714,...,50,50,57.1429,34.375,0,0,14.2857,33.3333,26.9231,50
CL3OP4,39,5.12821,38.4615,84.6154,100,64.1026,17.9487,0,0,35.8974,...,28.5714,100,86.6667,54.5455,15.3846,0,33.3333,0,12,28.5714


In [ ]:
df.rename(columns={"C": "C_W","A": "A_W","B": "B_W","D": "D_W","E": "E_W",},inplace = True)
df.rename(columns={'CLOP':'CLOP_W'},inplace=True)
df.drop(columns=['Open','High','Low','Close','Volume','range'],inplace = True)
df.drop(columns=['Reach1','Reach2','Reach3','Reach4','Reach6','Reach5'],inplace = True)
df.drop(columns=['RES2','RES3','RES4','RES5','RES6','SUP1','SUP2','SUP3','SUP4','SUP5'],inplace = True)
df.drop(columns=['SB1','SB2','RB1','RB2'],inplace = True)

In [ ]:
df_w=df.resample('1d').pad()

,C_W,D_W,B_W,E_W,A_W,CLOP_W
Date,,,,,,
2010-01-18,1135.916667,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
2010-01-19,1135.916667,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
2010-01-20,1135.916667,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
2010-01-21,1135.916667,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
2010-01-22,1135.916667,1146.166667,1125.666667,1156.416667,1115.416667,CL4OP3
...,...,...,...,...,...,...
2021-03-04,3842.416667,3917.166667,3767.666667,3991.916667,3692.916667,CL1OP3
2021-03-05,3842.416667,3917.166667,3767.666667,3991.916667,3692.916667,CL1OP3
2021-03-06,3842.416667,3917.166667,3767.666667,3991.916667,3692.916667,CL1OP3
